## Reddit Scraper

In [1]:
import os
import time
import csv
import praw
import json
import numpy as np
import pandas as pd

### Athentication

Follow steps in Reddit praw's quick start documentation

https://praw.readthedocs.io/en/latest/getting_started/quick_start.html

In [2]:
# Opening JSON file containing authentication info
with open('authentication.json', 'r') as openfile: 
    auth = json.load(openfile)

In [3]:
print('Authenticating...')
reddit_obj = praw.Reddit(client_id=auth["client_id"], client_secret=auth["client_secret"],
                         user_agent=auth["user_agent"],
                         username=auth["username"],
                         password=auth["password"])

print('Authenticated as /u/{}'.format(reddit_obj.user.me()))

Authenticating...
Authenticated as /u/cam_man_can


### Scrape posts

In [4]:
directory = "reddit_data\\"
subreddit_name = 'democrats'
headlines_file_path = directory + subreddit_name + '_posts.csv'

In [5]:
def scrape_posts(reddit, subreddit, headlines_file):
    """Scrapes 1000 posts from specified subreddit, logs post information into csv file"""
    
    if not os.path.exists(os.path.dirname(headlines_file)):
        os.makedirs(os.path.dirname(headlines_file))

    with open(headlines_file, "a",  encoding="utf-8") as outfile:
        for submission in reddit.subreddit(subreddit).top('all', limit=1000):
            # submission attributes: https://praw.readthedocs.io/en/latest/code_overview/models/submission.html
            print("\r \n", submission.title, end='')
            data = [
                submission.title,
                submission.author,
                submission.created_utc,
                submission.score,
                submission.domain,
                "%r" % submission.selftext,
                submission.id,
                submission.upvote_ratio,
                submission.num_comments
            ]
            writer = csv.writer(outfile)
            writer.writerow(data)
            # time.sleep(1)
                

In [6]:
scrape_posts(reddit_obj, subreddit_name, headlines_file_path)

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Giuliani associate wants to testify that Nunes aides hid Ukraine meetings on Biden dirt from Schiff

### Extract comments from scraped posts

In [7]:
comments_file_path = directory + subreddit_name + '_comments.csv'

In [8]:
def get_comments(reddit, headlines_file, comments_file):
    """Takes csv file containing post information, and scrapes the comments for each post, 
    saves comments and comment info to csv file"""
    
    if not os.path.exists(os.path.dirname(headlines_file)):
        os.makedirs(os.path.dirname(headlines_file))
        
    print("Fetching comments ...")
    with open(headlines_file, "r", encoding="utf-8") as infile:
        reader = csv.reader(infile)
        resume_flag = 0
        row_counter = 0

        for row in reader:
            print("\r post count:%s" % (row_counter), end='')
            
            if len(row) == 0:
                continue
            
            post_id = str(row[6])
            row_counter+=1

            if post_id == "6am00f":
                resume_flag = 1

            if resume_flag:
                continue

            submission = reddit.submission(id=post_id)
            submission.comments.replace_more(limit=30, threshold=10)

            comment_count = 0

            for comment in submission.comments.list():
                if comment_count >= 100:
                    break

                if isinstance(comment, praw.models.MoreComments):
                    comment_count += 1
                    continue

                comment_str = comment.body
                comment_author = comment.author
                comment_count += 1

                if comment_str == "[deleted]" or comment_str == "[removed]":
                    continue
                 
                with open(comments_file, "a",  encoding="utf-8") as outfile:
                    writer = csv.writer(outfile)
                    writer.writerow(["%r" % comment_str, comment_author, post_id])
            
        # time.sleep(1)

In [9]:
get_comments(reddit_obj, headlines_file_path, comments_file_path)

Fetching comments ...
 post count:996